<a href="https://colab.research.google.com/github/ssubin29/DaconAlone/blob/main/2021_ForecastingPriceContest/%5Bbaseline2%5D_LightGBM%EC%9D%84_%ED%99%9C%EC%9A%A9%ED%95%9C_%EA%B0%9C%EB%B3%84_%EB%AA%A8%EB%8D%B8%EB%A7%81(Public__0_367959).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Library Import & Settings

In [ ]:
import pandas as pd
import numpy as np
import lightgbm
from tqdm import tqdm
import warnings

In [ ]:
# 경고 끄기
pd.set_option('mode.chained_assignment', None)
warnings.filterwarnings(action='ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#data  = pd.read_csv("/content/drive/Shareddrives/2021_ForecastingPriceContest/dataset/public_data/train.csv")
data  = pd.read_csv("/content/drive/Shareddrives/2021_ForecastingPriceContest/output/merged_data.csv")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 전처리

## lag_feature 추가 및 기타 전처리

In [ ]:
def preprocessing(temp_df, pum, len_lag) :
    # p_lag(가격), q_lag(거래량) 추가 
    # lag 예측 변수의 시차
    for lag in range(1,len_lag+1) :
      temp_df[f'p_lag_{lag}'] = -1
      temp_df[f'q_lag_{lag}'] = -1
      for index in range(lag, len(temp_df)) :
        temp_df.loc[index, f'p_lag_{lag}'] = temp_df[f'{pum}_가격(원/kg)'][index-lag] #1일전, 2일전, ... 가격을 feature로 추가
        temp_df.loc[index, f'q_lag_{lag}'] = temp_df[f'{pum}_거래량(kg)'][index-lag] #1일전, 2일전, ... 거래량을 feature로 추가

    # month 추가
    temp_df['date'] = pd.to_datetime(temp_df['date'])
    temp_df['month'] = temp_df['date'].dt.month

    # 예측 대상(1w,2w,4w) 추가
    for week in ['1_week','2_week','4_week'] :
      temp_df[week] = 0
      n_week = int(week[0])
      for index in range(len(temp_df)) :
        try : temp_df[week][index] = temp_df[f'{pum}_가격(원/kg)'][index+7*n_week]
        except : continue

    # 불필요한 column 제거        
    temp_df = temp_df.drop(['date',f'{pum}_거래량(kg)',f'{pum}_가격(원/kg)'], axis=1)
    
    return temp_df

In [ ]:
train = data
train.head(2)

,date,요일,배추_거래량(kg),배추_가격(원/kg),무_거래량(kg),무_가격(원/kg),양파_거래량(kg),양파_가격(원/kg),건고추_거래량(kg),건고추_가격(원/kg),마늘_거래량(kg),마늘_가격(원/kg),대파_거래량(kg),대파_가격(원/kg),얼갈이배추_거래량(kg),얼갈이배추_가격(원/kg),양배추_거래량(kg),양배추_가격(원/kg),깻잎_거래량(kg),깻잎_가격(원/kg),시금치_거래량(kg),시금치_가격(원/kg),미나리_거래량(kg),미나리_가격(원/kg),당근_거래량(kg),당근_가격(원/kg),파프리카_거래량(kg),파프리카_가격(원/kg),새송이_거래량(kg),새송이_가격(원/kg),팽이버섯_거래량(kg),팽이버섯_가격(원/kg),토마토_거래량(kg),토마토_가격(원/kg),청상추_거래량(kg),청상추_가격(원/kg),백다다기_거래량(kg),백다다기_가격(원/kg),애호박_거래량(kg),애호박_가격(원/kg),캠벨얼리_거래량(kg),캠벨얼리_가격(원/kg),샤인마스캇_거래량(kg),샤인마스캇_가격(원/kg)
0,2016-01-01,금요일,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2016-01-02,토요일,80860.0,329.0,80272.0,360.0,122787.5,1281.0,3.0,11000.0,15019.0,5475.0,92334.0,1704.0,6359.0,1331.0,40028.0,348.0,4374.9,13242.0,16550.5,2339.0,10528.0,1729.0,13885.0,804.0,3853.0,3703.0,15797.0,2576.0,14634.0,1474.0,30950.0,1621.0,5125.0,9235.0,434.0,2109.0,19159.0,2414.0,880.0,2014.0,0.0,0.0


In [ ]:
# preprocessing 함수 예시
pum = '배추'
temp_df = train[['date',f'{pum}_거래량(kg)', f'{pum}_가격(원/kg)']]
# pum 별로 train
preprocessing(temp_df, pum, len_lag=28)

,p_lag_1,q_lag_1,p_lag_2,q_lag_2,p_lag_3,q_lag_3,p_lag_4,q_lag_4,p_lag_5,q_lag_5,p_lag_6,q_lag_6,p_lag_7,q_lag_7,p_lag_8,q_lag_8,p_lag_9,q_lag_9,p_lag_10,q_lag_10,p_lag_11,q_lag_11,p_lag_12,q_lag_12,p_lag_13,q_lag_13,p_lag_14,q_lag_14,p_lag_15,q_lag_15,p_lag_16,q_lag_16,p_lag_17,q_lag_17,p_lag_18,q_lag_18,p_lag_19,q_lag_19,p_lag_20,q_lag_20,p_lag_21,q_lag_21,p_lag_22,q_lag_22,p_lag_23,q_lag_23,p_lag_24,q_lag_24,p_lag_25,q_lag_25,p_lag_26,q_lag_26,p_lag_27,q_lag_27,p_lag_28,q_lag_28,month,1_week,2_week,4_week
0,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.0,-1.000000,-1.0,-1.000000,-1.0,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.0,-1.000000,-1.0,-1.000000,-1.0,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,1,420,449,625
1,0.000000,0.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.0,-1.000000,-1.0,-1.000000,-1.0,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.0,-1.000000,-1.0,-1.000000,-1.0,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,1,389,454,733
2,329.000000,80860.00,0.000000,0.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.0,-1.000000,-1.0,-1.000000,-1.0,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.0,-1.000000,-1.0,-1.000000,-1.0,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,1,0,0,1048
3,0.000000,0.00,329.000000,80860.00,0.000000,0.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.0,-1.000000,-1.0,-1.000000,-1.0,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.0,-1.000000,-1.0,-1.000000,-1.0,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,1,398,475,638
4,478.000000,1422742.50,0.000000,0.00,329.000000,80860.00,0.000000,0.00,-1.000000,-1.00,-1.000000,-1.0,-1.000000,-1.0,-1.000000,-1.0,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.0,-1.000000,-1.0,-1.000000,-1.0,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,-1.000000,-1.00,1,431,511,597
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2116,580.038536,1395747.15,538.959291,1592272.90,556.408045,1612247.30,531.337939,1570622.70,553.968214,1579252.10,559.218604,1696575.5,0.000000,0.0,609.211987,1054531.5,566.972458,1330920.55,441.969331,1517464.40,473.507965,1485657.10,515.905392,1370920.75,585.604100,1198007.70,0.000000,0.00,426.714740,921869.30,370.199348,1445591.20,330.633124,1672607.85,399.531962,1291821.70,381.107268,1333879.60,420.844809,1670250.10,0.000000,0.00,465.919195,849488.7,387.292546,1383823.5,484.783541,1255555.7,527.086078,1375062.36,580.921058,1406171.05,528.752722,1706354.80,0.000000,0.00,11,0,0,0
2117,612.637059,1754622.65,580.038536,1395747.15,538.959291,1592272.90,556.408045,1612247.30,531.337939,1570622.70,553.968214,1579252.1,559.218604,1696575.5,0.000000,0.0,609.211987,1054531.50,566.972458,1330920.55,441.969331,1517464.40,473.507965,1485657.10,515.905392,1370920.75,585.604100,1198007.70,0.000000,0.00,426.714740,921869.30,370.199348,1445591.20,

# 학습

## metric 정의

In [ ]:
def nmae(week_answer, week_submission):
    answer = week_answer.to_numpy()
    target_idx = np.where(answer!=0)
    true = answer[target_idx]
    pred = week_submission[target_idx]
    score = np.mean(np.abs(true-pred)/true)
    
    return score


def at_nmae(pred, dataset):
    y_true = dataset.get_label()
    week_1_answer = y_true[0::3]
    week_2_answer = y_true[1::3]
    week_4_answer = y_true[2::3]
    
    week_1_submission = pred[0::3]
    week_2_submission = pred[1::3]
    week_4_submission = pred[2::3]
    
    score1 = nmae(week_1_answer, week_1_submission)
    score2 = nmae(week_2_answer, week_2_submission)
    score4 = nmae(week_4_answer, week_4_submission)
    
    score = (score1+score2+score4)/3
    
    return 'score', score, False

## 학습 정의

In [ ]:
def model_train(x_train, y_train, x_valid, y_valid) :
    params = {'learning_rate': 0.01, 
              'max_depth': 6, 
              'boosting': 'gbdt', 
              'objective': 'regression',  
              'is_training_metric': True, 
              'num_leaves': 100, 
              'feature_fraction': 0.8, 
              'bagging_fraction': 0.8, 
              'bagging_freq': 5, 
              'seed':42,
              'num_threads':8
             }

    model = lightgbm.train(params, 
                   train_set = lightgbm.Dataset(data = x_train, label = y_train),
                   num_boost_round = 10000, 
                   valid_sets = lightgbm.Dataset(data = x_valid, label = y_valid), 
                   init_model = None, 
                   early_stopping_rounds = 100,
                   feval = at_nmae,
                   verbose_eval = False
                    )
    
    return model

## 품목 및 품종별 모델 학습

In [ ]:
unique_pum = [
    '배추', '무', '양파', '건고추','마늘',
    '대파', '얼갈이배추', '양배추', '깻잎',
    '시금치', '미나리', '당근',
    '파프리카', '새송이', '팽이버섯', '토마토',
]

unique_kind = [
    '청상추', '백다다기', '애호박', '캠벨얼리', '샤인마스캇'
]

In [ ]:
model_dict = {}
split = 28 #validation

for pum in tqdm(unique_pum + unique_kind):
    # 품목 품종별 전처리
    temp_df = train[['date',f'{pum}_거래량(kg)', f'{pum}_가격(원/kg)']]
    temp_df = preprocessing(temp_df, pum, len_lag=28)
    
    # 주차별(1,2,4w) 학습
    for week_num in [1,2,4] :
        x = temp_df[temp_df[f'{week_num}_week']>0].iloc[:,:-3]
        y = temp_df[temp_df[f'{week_num}_week']>0][f'{week_num}_week']
        
        #train, test split
        x_train = x[:-split]
        y_train = y[:-split]
        x_valid = x[-split:]
        y_valid = y[-split:]
        
        model_dict[f'{pum}_model_{week_num}'] = model_train(x_train, y_train, x_valid, y_valid)

100%|██████████| 21/21 [22:42<00:00, 64.88s/it]


# 추론

In [ ]:
submission = pd.read_csv('/content/drive/Shareddrives/2021_ForecastingPriceContest/dataset/sample_submission.csv')
public_date_list = submission[submission['예측대상일자'].str.contains('2020')]['예측대상일자'].str.split('+').str[0].unique()
# ['2020-09-29', ...]

for date in tqdm(public_date_list) :
    test = pd.read_csv(f'/content/drive/Shareddrives/2021_ForecastingPriceContest/output/merged_data.csv')
    for pum in unique_pum + unique_kind:
        # 예측기준일에 대해 전처리
        temp_test = pd.DataFrame([{'date' : date}]) #예측기준일
        alldata = pd.concat([train, test, temp_test], sort=False).reset_index(drop=True)
        alldata = alldata[['date', f'{pum}_거래량(kg)', f'{pum}_가격(원/kg)']].fillna(0)
        alldata = alldata.iloc[-28:].reset_index(drop=True)
        alldata = preprocessing(alldata, pum, len_lag=28)
        temp_test = alldata.iloc[-1].astype(float)
        
        # 개별 모델을 활용하여 1,2,4주 후 가격 예측
        for week_num in [1,2,4] :
            temp_model = model_dict[f'{pum}_model_{week_num}']
            result = temp_model.predict(temp_test)
            condition = (submission['예측대상일자']==f'{date}+{week_num}week')
            idx = submission[condition].index
            submission.loc[idx, f'{pum}_가격(원/kg)'] = result[0]



100%|██████████| 38/38 [05:12<00:00,  8.21s/it]


In [ ]:
submission.to_csv('baseline2_0920.csv',index=False, encoding='utf-8-sig')

In [ ]:
submission.head()

,예측대상일자,배추_가격(원/kg),무_가격(원/kg),양파_가격(원/kg),건고추_가격(원/kg),마늘_가격(원/kg),대파_가격(원/kg),얼갈이배추_가격(원/kg),양배추_가격(원/kg),깻잎_가격(원/kg),시금치_가격(원/kg),미나리_가격(원/kg),당근_가격(원/kg),파프리카_가격(원/kg),새송이_가격(원/kg),팽이버섯_가격(원/kg),토마토_가격(원/kg),청상추_가격(원/kg),백다다기_가격(원/kg),애호박_가격(원/kg),캠벨얼리_가격(원/kg),샤인마스캇_가격(원/kg)
0,2020-09-29+1week,860.429258,454.521041,883.035288,12309.907591,5959.544170,1515.121249,1123.529905,487.205442,7249.865662,2755.430745,4544.632746,878.309430,3196.886209,3137.029872,3020.451853,2481.196768,3475.201454,2074.803124,2105.248882,5010.701826,10773.141915
1,2020-09-29+2week,685.883972,498.832886,873.228427,12183.361637,6079.811245,1468.440842,1078.855623,496.483810,6455.305534,2453.362450,3315.671341,917.991467,3536.989920,3482.503867,2647.909047,2448.293052,4239.009105,1872.368587,1882.376800,4113.313915,10689.244407
2,2020-09-29+4week,690.358155,577.930608,880.415206,12400.285056,6485.178228,1366.131576,1094.101182,509.340041,5486.900843,2865.563235,2926.822735,949.702819,3455.335746,2898.608724,1935.198424,2345.182735,3378.905894,1861.493707,1877.351924,4110.823237,9109.397096
3,2020-09-30+1week,860.429258,454.521041,883.035288,12309.907591,5959.544170,1515.121249,1123.529905,487.205442,7249.865662,2755.430745,4544.632746,878.309430,3196.886209,3137.029872,3020.451853,2481.196768,3475.201454,2074.803124,2105.248882,5010.701826,10773.141915
4,2020-09-30+2week,685.883972,498.832886,873.228427,12183.361637,6079.811245,1468.440842,1078.855623,496.483810,6455.305534,2453.362450,3315.671341,917.991467,3536.989920,3482.503867,2647.909047,2448.293052,4239.009105,1872.368587,1882.376800,4113.313915,10689.244407


In [ ]:
!pip install dacon_submit_api-0.0.4-py3-none-any.whl

from dacon_submit_api import dacon_submit_api 

result = dacon_submit_api.post_submission_file(
    'baseline2_0920.csv', 
    '개인 Token', 
    '235801', 
    'DACONIO', 
    'DACON_Baseline'
)

Processing ./dacon_submit_api-0.0.4-py3-none-any.whl
{'isSubmitted': False, 'detail': 'Over max submission count of Daily. 일일 제출 가능한 최대 횟수가 초과 되었습니다.'}
